In [ ]:
import numpy as np
import pandas as pd
import pymc as pm
import matplotlib.pyplot as plt
import aesara.tensor as at
import arviz as az
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from scipy import stats
import scipy
import warnings

from deepemogp import feature_extractor
from deepemogp.signal import physio as physio
from deepemogp import datasets as datasets
from deepemogp.signal import behavior as behavior

In [ ]:
warnings.simplefilter(action="ignore", category=FutureWarning)
scaler = StandardScaler()

In [ ]:
i = subject = 45

In [ ]:
TRIAL = 160
num_trials_to_remove = 48
K = 5

In [ ]:
hr = pd.read_csv('data/features/hr/'+str(i)+'.csv')
hr = hr[num_trials_to_remove:]

eda = pd.read_csv('data/features/eda/' + str(i) + '.csv')
eda = eda[num_trials_to_remove:]

pupil = pd.read_csv('data/features/pupil/' + str(i) + '.csv')
pupil = pupil[num_trials_to_remove:]

In [ ]:
csv_ = '/home/paolo/matteo/matteo/unimi/tesi_master/code/osfstorage-archive/behavior/LookAtMe_045.csv'
global_data = pd.read_csv(csv_, sep='\t')
y = np.array(list([int(d > 2) for d in global_data['rating']]))
e_labels = y[:, np.newaxis]  # rating > 2
e_labels = e_labels[num_trials_to_remove:]

In [ ]:
N_pupil = pupil.shape[0]
D_pupil = pupil.shape[1]

N_hr = hr.shape[0]
D_hr = hr.shape[1]

N_eda = eda.shape[0]
D_eda = eda.shape[1]

N_e = e_labels.shape[0]
D_e = e_labels.shape[1]


print(N_pupil,D_pupil)
print(N_hr,D_hr)
print(N_eda,D_eda)
print(N_e, D_e)

In [ ]:
hr = hr.to_numpy()
eda = eda.to_numpy()
pupil = pupil.to_numpy()

In [ ]:
trial_i = 110
index = np.arange(60)
hr_trial = hr[trial_i]
eda_trial = eda[trial_i]

In [ ]:
hr_trial.shape

In [ ]:
eda_trial.shape

In [ ]:
coords_ = {'time':index, 'hr_': hr_trial, 'eda_':eda_trial}

In [ ]:
coords_['time']

In [ ]:
with pm.Model(coords=coords_) as rolling:
    sigma_alpha = pm.Exponential("sigma_alpha", 50.0)
    sigma_beta = pm.Exponential("sigma_beta", 50.0)

    alpha = pm.GaussianRandomWalk(
        "alpha", sigma=sigma_alpha, init_dist=pm.Normal.dist(0, 10), dims="time"
    )
    beta = pm.GaussianRandomWalk(
        "beta", sigma=sigma_beta, init_dist=pm.Normal.dist(0, 10), dims="time"
    )
    regression = alpha + beta * hr_trial

    # Assume eda data are Normally distributed, the mean comes from the regression.
    sd = pm.HalfNormal("sd", sigma=0.1)
    likelihood = pm.Normal("y", mu=regression, sigma=sd, observed=eda_trial)

In [ ]:
graph_rolling = pm.model_to_graphviz(rolling)
graph_rolling.view('rolling_regr')

In [ ]:
with rolling:
    approx = pm.fit(100000, callbacks=[pm.callbacks.CheckParametersConvergence(tolerance=1e-4)])
    trace = approx.sample(500)

In [ ]:
with rolling:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123)

In [ ]:
eda_pred = posterior_predictive.posterior_predictive['y']
eda_pred

In [ ]:
eda_trial

In [ ]:
eda_pred[0]

In [ ]:
plt.plot(eda_trial)

In [ ]:
np.squeeze(eda_pred.mean('draw',keepdims='false')[0])

In [ ]:
np.squeeze(np.mean(eda_pred[0]0)[0])[:, np.newaxis]

In [ ]:
az.plot_ppc(posterior_predictive,)

In [ ]:
eda_pred_mean = np.squeeze(stats.mode(eda_pred[0], keepdims=False)[0])[:,np.newaxis]

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

mean_squared_error(eda, eda_pred_mean)

In [ ]:
mean_absolute_error(coords_['eda_'], eda_pred_mean)

In [ ]:
final_df = pd.DataFrame()
